# VGG-16 on tf2
VGG-16 is a convolutional neural network architecture, it’s name VGG-16 comes from the fact that it has 16 layers. It’s layers consists of Convolutional layers, Max Pooling layers, Activation layers, Fully connected layers.

Lets start by importing the necessary modules

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL

Great now lets start building the VGG model, but first lets explore the architecture of th model, the special thing about VGG model is that it only uses a 3x3 kernel and follows a pattern and this seems to work :D 

<img src="images\VGG-16.png">

There are 13 convolutional layers, 5 Max Pooling layers and 3 Dense layers which sums up to 21 layers but only 16 weight layers.

Conv 1 has number of filters as 64 while Conv 2 has 128 filters, Conv 3 has 256 filters while Conv 4 and Conv 5 has 512 filters.


# Okay now lets build the model !! 

In [2]:
# importing the necessary moduels 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

In [3]:
# VGG-16 Model
class VGG16(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
    
        #conv layers (conv1)
        self.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', input_shape=input_shape, activation='relu'))
        self.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
    
        #maxpooling
        self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
        #conv layers (conv2) 
        self.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
        self.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
             
        #maxpooling
        self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
        #conv layers (conv3)
        self.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
        self.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
        self.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    
        #maxpooling
        self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
        #conv layers (conv4)
        self.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))        
        self.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
        self.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    
        #maxpooling
        self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
        #conv layers (conv5)
        self.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))        
        self.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
        self.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    
        #maxpooling
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
             
        self.add(Flatten())
    
        #FC layers
        self.add(Dense(units=4096,activation="relu"))
        self.add(Dense(units=4096,activation="relu"))
             
        #outputlayer
        self.add(Dense(units=num_classes, activation="softmax"))
    
        self.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [16]:
num_classes = 2
model = VGG16((224, 224, 3), num_classes)

In [17]:
model.summary()

Model: "vg_g16_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 256)       295

## Lets train it on some data

In [6]:
EPOCHS = 3
BATCH_SIZE = 32
image_height = 224
image_width = 224
train_dir = "data/training_set"
valid_dir = "data/test_set"
model_dir = "alex_net.h5"

##  I will use a simple Dogs v Cats Classifier here
#### The source of the data : <https://www.kaggle.com/tongpython/cat-and-dog>

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=10,
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  shear_range=0.1,
                  zoom_range=0.1)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    #batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="binary")

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    #batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode="binary"
                                                    )
train_num = train_generator.samples
valid_num = valid_generator.samples

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [19]:
#Callbacks

checkpoint = tf.keras.callbacks.ModelCheckpoint('alexnet.h5', monitor='val_loss', verbose=0, save_best_only=True,
                                                save_weights_only=False, mode='auto', save_freq='epoch')

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

callback_list = [checkpoint]

In [20]:
# start training
hist = model.fit(train_generator,
                    epochs=EPOCHS,
                    #steps_per_epoch=train_num // BATCH_SIZE,
                    validation_data=valid_generator,
                    #validation_steps=valid_num // BATCH_SIZE,
                    callbacks=callback_list,
                    verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 251 steps, validate for 64 steps
Epoch 1/3
  1/251 [..............................] - ETA: 49:26WARNING:tensorflow:Can save best model only with val_loss available, skipping.


ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill] name: Adam/dense_3/kernel/v/Initializer/zeros/

### Okayyyy, I do not have enough memory in my GPU to train 130m Parameters, soo use pretrained weights
### - Demonstrated in another notebook